In [65]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk.data
import collections

In [66]:
filenames = ["data/reviews_Home_and_Kitchen_5.json"]
filename = "data/reviews_Home_and_Kitchen_5.json"
file_name = "rawdata.pkl"
adversarial_file_name = "adversialdata.pkl"
max_features = 20000
maxlen = 100
batch_size = 32
num_class = 2

In [105]:
data = pd.read_pickle(file_name)

In [68]:
data.head(10)

,overall,reviewText
489257,1,I use this set for both general food storage i...
318868,0,Not a huge fan. I like to have a side for damp...
527578,1,I have always wanted to have a French press at...
102291,0,I purchased this to replace a breadmaker that ...
137178,0,"I bought this pan because my 12"" pan needed to..."
233040,0,"Just sayin'. I mean, honestly. Do we really ne..."
413128,1,"This rack fits the sink perfectly, and isn't b..."
149988,0,I bought one of the 20X72 inch mats for my wif...
18029,1,I have four different sizes of these scoops an...
129849,1,These replacement rubbers are perfect for use ...


### edit datafram

In [73]:
POS_TO_WORDNET = {
    'NN': wn.NOUN
#     'JJ': wn.ADJ,
#     'JJR': wn.ADJ,
#     'JJS': wn.ADJ,
}

In [74]:
def alter_wordnet_antonyms(token):
    if token[1] not in POS_TO_WORDNET: return None
    w = token[0].lower()
    wn_pos = POS_TO_WORDNET[token[1]]
    synsets = wn.synsets(w, wn_pos)
    if not synsets: return None
    synset = synsets[0]
    antonyms = []
    for lem in synset.lemmas():
        if lem.antonyms():
            for a in lem.antonyms():
                new_word = a.name()
                if '_' in a.name(): continue
                antonyms.append(new_word)
    return antonyms

In [75]:
texts = data['reviewText']
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [106]:
index = 0
for i, row in data.iterrows():
    text = texts[i]
    tokenized = tokenizer.tokenize(text)
    words = word_tokenize(text)
    tokens = nltk.pos_tag(words)
    output = ""
    count = 0
    for token in tokens:
        if token == '.':
            output = output + token
            index = 0
            break
        if index != 0:
            output = output + ' '
        antonyms = alter_wordnet_antonyms(token)
        if antonyms is None or len(antonyms) == 0:
            output = output + token[0]
        else:
            print(i, ": change from ", token[0], " to ", antonyms[0])
            output = output + antonyms[0]
        index = index + 1
    data.at[i,'reviewText'] = output

102291 : change from  outside  to  inside
149988 : change from  wife  to  husband
180587 : change from  color  to  colorlessness
315837 : change from  attention  to  inattention
428417 : change from  ability  to  inability
428417 : change from  soundness  to  unsoundness
531813 : change from  capacity  to  incapacity
531813 : change from  power  to  powerlessness
531813 : change from  power  to  powerlessness
531813 : change from  power  to  powerlessness
531813 : change from  ability  to  inability
396739 : change from  outside  to  inside
469813 : change from  attention  to  inattention
194455 : change from  color  to  colorlessness
194455 : change from  color  to  colorlessness
163452 : change from  safety  to  danger
163452 : change from  future  to  past
64592 : change from  husband  to  wife
97783 : change from  danger  to  safety
97783 : change from  daughter  to  son
513349 : change from  difference  to  sameness
94965 : change from  color  to  colorlessness
469100 : change fro

418361 : change from  difference  to  sameness
256554 : change from  love  to  hate
446704 : change from  thickness  to  thinness
86345 : change from  power  to  powerlessness
403383 : change from  daughter  to  son
403383 : change from  husband  to  wife
403383 : change from  daughter  to  son
435602 : change from  outside  to  inside
249744 : change from  power  to  powerlessness
249744 : change from  husband  to  wife
249744 : change from  power  to  powerlessness
219568 : change from  safety  to  danger
444445 : change from  depression  to  elation
212695 : change from  daughter  to  son
107271 : change from  wife  to  husband
107271 : change from  experience  to  inexperience
539176 : change from  past  to  future
223092 : change from  color  to  colorlessness
144300 : change from  repute  to  disrepute
434652 : change from  daughter  to  son
481383 : change from  outside  to  inside
242940 : change from  conception  to  misconception
77748 : change from  softness  to  hardness
77

405948 : change from  wife  to  husband
405948 : change from  wife  to  husband
492375 : change from  color  to  colorlessness
364224 : change from  inside  to  outside
233726 : change from  satisfaction  to  dissatisfaction
123172 : change from  daughter  to  son
123172 : change from  mother  to  father
123172 : change from  daughter  to  son
25339 : change from  mother  to  father
522518 : change from  future  to  past
41041 : change from  husband  to  wife
86999 : change from  yea  to  nay
410847 : change from  man  to  woman
410847 : change from  comfort  to  discomfort
119534 : change from  employee  to  employer
215610 : change from  comfort  to  discomfort
460013 : change from  color  to  colorlessness
460013 : change from  outside  to  inside
450080 : change from  wife  to  husband
75273 : change from  inside  to  outside
75273 : change from  inside  to  outside
480426 : change from  color  to  colorlessness
333156 : change from  death  to  birth
68599 : change from  mother  to

413038 : change from  powerlessness  to  power
269970 : change from  daughter  to  son
269970 : change from  waking  to  sleeping
364975 : change from  powerlessness  to  power
364975 : change from  powerlessness  to  power
131102 : change from  colorlessness  to  color
390229 : change from  powerlessness  to  power
383482 : change from  inside  to  outside
323644 : change from  thinness  to  thickness
285946 : change from  wife  to  husband
444998 : change from  outside  to  inside
112833 : change from  incaution  to  caution
340883 : change from  unconnectedness  to  connectedness
197846 : change from  rear  to  head
197846 : change from  wife  to  husband
383122 : change from  powerlessness  to  power
497609 : change from  outside  to  inside
240067 : change from  man  to  woman
89860 : change from  inability  to  ability
501427 : change from  colorlessness  to  color
468303 : change from  daughter  to  son
152213 : change from  powerlessness  to  power
189621 : change from  certain

129360 : change from  past  to  future
129360 : change from  husband  to  wife
427627 : change from  daughter  to  son
93450 : change from  nondrinker  to  drinker
93450 : change from  nondrinker  to  drinker
93450 : change from  nondrinker  to  drinker
406920 : change from  inattention  to  attention
378577 : change from  colorlessness  to  color
147409 : change from  colorlessness  to  color
147409 : change from  instability  to  stability
147409 : change from  colorlessness  to  color
434513 : change from  outside  to  inside
107027 : change from  outgo  to  income
85487 : change from  danger  to  safety
85487 : change from  powerlessness  to  power
125253 : change from  son  to  daughter
125253 : change from  sameness  to  difference
338662 : change from  wife  to  husband
243459 : change from  yes  to  no
125500 : change from  inexperience  to  experience
64086 : change from  powerlessness  to  power
116939 : change from  wife  to  husband
133148 : change from  illness  to  wellne

411521 : change from  past  to  future
114904 : change from  inexperience  to  experience
438645 : change from  sameness  to  difference
409649 : change from  colorlessness  to  color
398055 : change from  colorlessness  to  color
201530 : change from  inexperience  to  experience
22645 : change from  colorlessness  to  color
22645 : change from  colorlessness  to  color
367637 : change from  colorlessness  to  color
367637 : change from  inattention  to  attention
367637 : change from  rear  to  head
298999 : change from  sameness  to  difference
192044 : change from  inside  to  outside
192044 : change from  husband  to  wife
426075 : change from  wife  to  husband
277416 : change from  colorlessness  to  color
493739 : change from  powerlessness  to  power
331882 : change from  wife  to  husband
73132 : change from  rear  to  head
19840 : change from  husband  to  wife
19840 : change from  husband  to  wife
214904 : change from  colorlessness  to  color
214904 : change from  colorle

517452 : change from  daughter  to  son
272458 : change from  father  to  mother
285257 : change from  inattention  to  attention
420148 : change from  inside  to  outside
420148 : change from  woman  to  man
250140 : change from  nonbeing  to  being
310580 : change from  disloyalty  to  loyalty
276079 : change from  rear  to  head
184897 : change from  sameness  to  difference
268940 : change from  wife  to  husband
268940 : change from  inside  to  outside
481244 : change from  colorlessness  to  color
535438 : change from  thinness  to  thickness
468099 : change from  man  to  woman
37092 : change from  outside  to  inside
373041 : change from  incapacity  to  capacity
528806 : change from  birth  to  death
528806 : change from  sameness  to  difference
528806 : change from  powerlessness  to  power
528806 : change from  sameness  to  difference
528806 : change from  inexperience  to  experience
99334 : change from  colorlessness  to  color
165459 : change from  anticyclone  to  cyc

KeyboardInterrupt: 

In [104]:
data.at[i,'reviewText']

' I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the outside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! ) .'

In [98]:
data.at[102291,'reviewText'] = " I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the inside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! ) ."

In [99]:
data.at[102291,'reviewText']

' I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the inside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! ) .'

In [78]:
data.to_pickle(adversarial_file_name)

In [85]:
data['reviewText'][102291]

' I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the outside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! ) .'

In [87]:
texts[102291]

' I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the outside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! ) .'

In [90]:
data.at[i,'reviewText'] = " I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the inside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! )."

In [91]:
data['reviewText'][102291]

' I purchased this to replace a breadmaker that was getting old . I am very disappointed ! On the LIGHT setting , the bread is overdone - with a very dark and hard crust . It does not mix the ingredients well , as there are bits of dough and loose particles of ingredients along the outside of the loaf . I would not recommend this item to anyone . I will not be buying any more Sunbeam breadmakers . Big mistake . I miss my old breadmaker , and will be looking for another one . This one is going to Goodwill ( so be careful what you buy there ! ) .'

In [72]:
wn.synsets('dog') # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [73]:
wn.synsets('dog', pos=wn.VERB)

[Synset('chase.v.01')]

In [74]:
wn.synset('dog.n.01')

Synset('dog.n.01')

In [75]:
print(wn.synset('dog.n.01').definition())

a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds


In [76]:
len(wn.synset('dog.n.01').examples())

1

In [77]:
print(wn.synset('dog.n.01').examples()[0])

the dog barked all night


In [78]:
wn.synset('dog.n.01').lemmas()

[Lemma('dog.n.01.dog'),
 Lemma('dog.n.01.domestic_dog'),
 Lemma('dog.n.01.Canis_familiaris')]

In [79]:
[str(lemma.name()) for lemma in wn.synset('dog.n.01').lemmas()]

['dog', 'domestic_dog', 'Canis_familiaris']

In [80]:
wn.lemma('dog.n.01.dog').synset()

Synset('dog.n.01')

In [85]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vwslz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True